In [4]:
from pyLCIO import IOIMPL
from tqdm.auto import tqdm
from glob import glob
import os.path as osp
import numpy as np

In [16]:
reader = IOIMPL.LCFactory.getInstance().createLCReader()
reader.open(osp.abspath("zh.slcio"))
result = []

for event in (pbar := tqdm(range(reader.getNumberOfEvents()))):
    event = reader.readNextEvent()
    pbar.set_description(f'Processing event {event.getEventNumber()}')

    # Iterate through the MCParticle collection and find muon and anti muon
    # see page 5 on https://pdg.lbl.gov/2019/reviews/rpp2019-rev-monte-carlo-numbering.pdf
    iso_leptons = event.getCollection('ISOLeptons')
    
    E = 0
    pX = 0
    pY = 0
    pZ = 0
    n_muons = 0
    
    for iso_lepton in iso_leptons:
        if abs(iso_lepton.getType()) == 13:
            mom = iso_lepton.getMomentum()
            
            E += iso_lepton.getEnergy()
            pX += mom[0]
            pY += mom[1]
            pZ += mom[2]
            n_muons += 1

    if n_muons == 2:
        result.append([E, pX, pY, pZ])
        
    # break

reader.getNumberOfEvents()

result = np.array(result)

  0%|          | 0/9400 [00:00<?, ?it/s]

In [14]:
reader.getNumberOfEvents()

9400

In [17]:
len(result)

8355

In [18]:
np.save('mumuH_250.npy', result)